# API FOOTBALL

## Estrazione Squadre e Giocatori da una Competizione

In questo notebook, useremo l'API di API-SPORTS per ottenere tutte le squadre e i giocatori di una competizione. In particolare, ci focalizzeremo sulla Premier League inglese per la stagione 2021-2022.

Dall'esempio [how-to-get-all-teams-and-players-from-a-league-id](https://www.api-football.com/news/post/how-to-get-all-teams-and-players-from-a-league-id)

In [52]:
# Importare le Librerie
import requests
import time
import json
import pandas as pd

In [53]:
from dotenv import load_dotenv
import os

# Carica le variabili d'ambiente dal file .env
load_dotenv()
RAPIDAPI_KEY = os.environ['API_KEY']


In [65]:
# Funzioni per chiamare l'API
# def call_api(endpoint, params=None):
#     url = f'https://v3.football.api-sports.io/{endpoint}'
#     headers = {
#         'x-rapidapi-key': RAPIDAPI_KEY 
#     }
#     response = requests.get(url, headers=headers, params=params)
#     return response.json()


def call_api(endpoint, params=None):
    url = f'https://v3.football.api-sports.io/{endpoint}'
    headers = {
        'x-rapidapi-key': RAPIDAPI_KEY 
    }
    
    try:
        response = requests.get(url, headers=headers, params=params)
        response.raise_for_status()  # Raise an error for bad status codes
        
        # Stampa gli header di rate limiting
        rate_limit_headers = {
            'x-ratelimit-requests-limit': response.headers.get('x-ratelimit-requests-limit'),
            'x-ratelimit-requests-remaining': response.headers.get('x-ratelimit-requests-remaining'),
            'X-RateLimit-Limit': response.headers.get('X-RateLimit-Limit'),
            'X-RateLimit-Remaining': response.headers.get('X-RateLimit-Remaining')
        }
        for key, value in rate_limit_headers.items():
            print(f'{key}: {value}')
        
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f'An error occurred: {e}')
        return None

In [89]:
# funzione ricorsiva per ottenere i dati dei giocatori
def fetch_players_data(league, season, page=1, players_data=[]):
    params = {
        'league': league,
        'season': season,
        'page': page
    }
    players = call_api('players', params)
    players_data.extend(players['response'])

    if players['paging']['current'] < players['paging']['total']:
        page += 1
        time.sleep(6.5)  # Pausa di 10 secondo per evitare di superare il rate limit (10 req / min)
        return fetch_players_data(league, season, page, players_data)
    
    return players_data

### Estrazione delle squadre

In [41]:
# Ottenere Tutte le Squadre della Competizione
# Utilizziamo la funzione `call_api` per ottenere tutte le squadre della Serie A per la stagione 2023-2024
# parametri: league_id = 135, season = 2023

teams = call_api('teams', {'league': 135, 'season': 2023})

In [47]:
#Esempio team
print(json.dumps(teams["response"][1], indent=4))

{
    "team": {
        "id": 488,
        "name": "Sassuolo",
        "code": "SAS",
        "country": "Italy",
        "founded": 1922,
        "national": false,
        "logo": "https://media.api-sports.io/football/teams/488.png"
    },
    "venue": {
        "id": 935,
        "name": "MAPEI Stadium - Citt\u00e0 del Tricolore",
        "address": "Piazza Azzuri d&apos;Italia, 1",
        "city": "Reggio Emilia",
        "capacity": 23717,
        "surface": "grass",
        "image": "https://media.api-sports.io/football/venues/935.png"
    }
}


In [45]:
# Salvo i teams in file json
with open('./data/teams.json', 'w', encoding='utf-8') as f:
    json.dump(teams["response"], f, ensure_ascii=False, indent=1)

In [8]:
# Creo il DF per esplorare i dati
# Estrarre la parte di response
teams_response = teams['response']

# Creare una lista di dizionari con le informazioni della squadra
teams_list = []
for team in teams_response:
    team_info = {
        'team_id': team['team']['id'],
        'team_name': team['team']['name'],
        'team_code': team['team']['code'],
        'team_country': team['team']['country'],
        'team_founded': team['team']['founded'],
        'team_national': team['team']['national'],
        'team_logo': team['team']['logo'],
        'venue_id': team['venue']['id'],
        'venue_name': team['venue']['name'],
        'venue_address': team['venue']['address'],
        'venue_city': team['venue']['city'],
        'venue_capacity': team['venue']['capacity'],
        'venue_surface': team['venue']['surface'],
        'venue_image': team['venue']['image']
    }
    teams_list.append(team_info)

# Convertire la lista di dizionari in un DataFrame
teams_df = pd.DataFrame(teams_list)

# Visualizzare il DataFrame
teams_df

,team_id,team_name,team_code,team_country,team_founded,team_national,team_logo,venue_id,venue_name,venue_address,venue_city,venue_capacity,venue_surface,venue_image
0,487,Lazio,LAZ,Italy,1900,False,https://media.api-sports.io/football/teams/487...,910,Stadio Olimpico,"Viale dei Gladiatori, 2 / Via del Foro Italico",Roma,68530,grass,https://media.api-sports.io/football/venues/91...
1,488,Sassuolo,SAS,Italy,1922,False,https://media.api-sports.io/football/teams/488...,935,MAPEI Stadium - Città del Tricolore,"Piazza Azzuri d&apos;Italia, 1",Reggio Emilia,23717,grass,https://media.api-sports.io/football/venues/93...
2,489,AC Milan,MIL,Italy,1899,False,https://media.api-sports.io/football/teams/489...,907,Stadio Giuseppe Meazza,Via Piccolomini 5,Milano,80018,grass,https://media.api-sports.io/football/venues/90...
3,490,Cagliari,CAG,Italy,1920,False,https://media.api-sports.io/football/teams/490...,12275,Unipol Domus,Via Raimondo Carta Raspi,Cagliari,16416,grass,https://media.api-sports.io/football/venues/12...
4,492,Napoli,NAP,Italy,1904,False,https://media.api-sports.io/football/teams/492...,11904,Stadio Diego Armando Maradona,Pizzale Vincenzo Tecchio,Napoli,60240,grass,https://media.api-sports.io/football/venues/11...
5,494,Udinese,UDI,Italy,1896,False,https://media.api-sports.io/football/teams/494...,20416,Bluenergy Stadium,"Piazza le Repubblica Argentina, 3",Udine,25952,grass,https://media.api-sports.io/football/venues/20...
6,495,Genoa,GEN,Italy,1893,False,https://media.api-sports.io/football/teams/495...,905,Stadio Comunale Luigi Ferraris,"Via Giovanni De Prà, 1",Genova,36703,grass,https://media.api-sports.io/football/venues/90...
7,496,Juventus,JUV,Italy,1897,False,https://media.api-sports.io/football/teams/496...,909,Allianz Stadium,Strada Comunale di Altessano 131,Torino,45666,grass,https://media.api-sports.io/football/venues/90...
8,497,AS Roma,ROM,Italy,1927,False,https://media.api-sports.io/football/teams/497...,910,Stadio Olimpico,"Viale dei Gladiatori, 2 / Via del Foro Italico",Roma,68530,grass,https://media.api-sports.io/football/venues/91...
9,499,Atalanta,ATA,Italy,1907,False,https://media.api-sports.io/football/teams/499...,879,Gewiss Stadium,Viale Giulio Cesare 18,Bergamo,21300,grass,https://media.api-sports.io/football/venues/87...


### Estrazione dei Giocatori

In [90]:
players_list = fetch_players_data(league=135, season=2023)

x-ratelimit-requests-limit: 100
x-ratelimit-requests-remaining: 39
X-RateLimit-Limit: 10
X-RateLimit-Remaining: 9
x-ratelimit-requests-limit: 100
x-ratelimit-requests-remaining: 38
X-RateLimit-Limit: 10
X-RateLimit-Remaining: 8
x-ratelimit-requests-limit: 100
x-ratelimit-requests-remaining: 37
X-RateLimit-Limit: 10
X-RateLimit-Remaining: 7
x-ratelimit-requests-limit: 100
x-ratelimit-requests-remaining: 36
X-RateLimit-Limit: 10
X-RateLimit-Remaining: 6
x-ratelimit-requests-limit: 100
x-ratelimit-requests-remaining: 35
X-RateLimit-Limit: 10
X-RateLimit-Remaining: 5
x-ratelimit-requests-limit: 100
x-ratelimit-requests-remaining: 34
X-RateLimit-Limit: 10
X-RateLimit-Remaining: 4
x-ratelimit-requests-limit: 100
x-ratelimit-requests-remaining: 39
X-RateLimit-Limit: 10
X-RateLimit-Remaining: 9
x-ratelimit-requests-limit: 100
x-ratelimit-requests-remaining: 38
X-RateLimit-Limit: 10
X-RateLimit-Remaining: 8
x-ratelimit-requests-limit: 100
x-ratelimit-requests-remaining: 37
X-RateLimit-Limit: 10

In [91]:
# numero giocatori
len(players_list)

1011

In [92]:
# Esempio player in json
print(json.dumps(players_list[1], indent=4))

{
    "player": {
        "id": 2897,
        "name": "Kim Min-Jae",
        "firstname": "Min-Jae",
        "lastname": "Kim",
        "age": 27,
        "birth": {
            "date": "1996-11-15",
            "place": "Tongyeong",
            "country": "Korea Republic"
        },
        "nationality": "Korea Republic",
        "height": "190 cm",
        "weight": "84 kg",
        "injured": false,
        "photo": "https://media.api-sports.io/football/players/2897.png"
    },
    "statistics": [
        {
            "team": {
                "id": 492,
                "name": "Napoli",
                "logo": "https://media.api-sports.io/football/teams/492.png"
            },
            "league": {
                "id": 135,
                "name": "Serie A",
                "country": "Italy",
                "logo": "https://media.api-sports.io/football/leagues/135.png",
                "flag": "https://media.api-sports.io/flags/it.svg",
                "season": 2023
       

In [93]:
# Salvataggio giocatori in file json
with open('./data/players.json', 'w', encoding='utf-8') as f:
    json.dump(players_list, f, ensure_ascii=False, indent=1)

In [98]:
# Carico i giocatori dal file json
with open('./data/players.json', "r", encoding="utf-8") as f:
    players_list = json.load(f)

In [99]:
# Creo Dataframe per esplorare i dati
players_info_list = []
for player_entry in players_list:
    player = player_entry['player']
    for stat in player_entry['statistics']:
        player_info = {
            'player_id': player['id'],
            'player_name': player['name'],
            'player_firstname': player['firstname'],
            'player_lastname': player['lastname'],
            'player_age': player['age'],
            'player_birth_date': player['birth']['date'],
            'player_birth_place': player['birth']['place'],
            'player_birth_country': player['birth']['country'],
            'player_nationality': player['nationality'],
            'player_height': player['height'],
            'player_weight': player['weight'],
            'player_injured': player['injured'],
            'player_photo': player['photo'],
            'team_id': stat['team']['id'],
            'team_name': stat['team']['name'],
            'team_logo': stat['team']['logo'],
            'league_id': stat['league']['id'],
            'league_name': stat['league']['name'],
            'league_country': stat['league']['country'],
            'league_logo': stat['league']['logo'],
            'league_flag': stat['league']['flag'],
            'season': stat['league']['season'],
            'games_appearences': stat['games']['appearences'],
            'games_lineups': stat['games']['lineups'],
            'games_minutes': stat['games']['minutes'],
            'games_number': stat['games']['number'],
            'games_position': stat['games']['position'],
            'games_rating': stat['games']['rating'],
            'games_captain': stat['games']['captain'],
            'substitutes_in': stat['substitutes']['in'],
            'substitutes_out': stat['substitutes']['out'],
            'substitutes_bench': stat['substitutes']['bench'],
            'shots_total': stat['shots']['total'],
            'shots_on': stat['shots']['on'],
            'goals_total': stat['goals']['total'],
            'goals_conceded': stat['goals']['conceded'],
            'goals_assists': stat['goals']['assists'],
            'goals_saves': stat['goals']['saves'],
            'passes_total': stat['passes']['total'],
            'passes_key': stat['passes']['key'],
            'passes_accuracy': stat['passes']['accuracy'],
            'tackles_total': stat['tackles']['total'],
            'tackles_blocks': stat['tackles']['blocks'],
            'tackles_interceptions': stat['tackles']['interceptions'],
            'duels_total': stat['duels']['total'],
            'duels_won': stat['duels']['won'],
            'dribbles_attempts': stat['dribbles']['attempts'],
            'dribbles_success': stat['dribbles']['success'],
            'dribbles_past': stat['dribbles']['past'],
            'fouls_drawn': stat['fouls']['drawn'],
            'fouls_committed': stat['fouls']['committed'],
            'cards_yellow': stat['cards']['yellow'],
            'cards_yellowred': stat['cards']['yellowred'],
            'cards_red': stat['cards']['red'],
            'penalty_won': stat['penalty']['won'],
            'penalty_commited': stat['penalty']['commited'],
            'penalty_scored': stat['penalty']['scored'],
            'penalty_missed': stat['penalty']['missed'],
            'penalty_saved': stat['penalty']['saved']
        }
        players_info_list.append(player_info)

# Convertire la lista di dizionari in un DataFrame
players_df = pd.DataFrame(players_info_list)

# Visualizzare il DataFrame
players_df

,player_id,player_name,player_firstname,player_lastname,player_age,player_birth_date,player_birth_place,player_birth_country,player_nationality,player_height,...,fouls_drawn,fouls_committed,cards_yellow,cards_yellowred,cards_red,penalty_won,penalty_commited,penalty_scored,penalty_missed,penalty_saved
0,188,S. Handanovič,Samir,Handanovič,39.0,1984-07-14,Ljubljana,Slovenia,Slovenia,193 cm,...,NaN,NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN
1,2897,Kim Min-Jae,Min-Jae,Kim,27.0,1996-11-15,Tongyeong,Korea Republic,Korea Republic,190 cm,...,NaN,NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN
2,30786,S. Sturaro,Stefano,Sturaro,30.0,1993-03-09,Sanremo,Italy,Italy,181 cm,...,NaN,NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN
3,30974,A. Cordaz,Alex,Cordaz,40.0,1983-01-01,Vittorio Veneto,Italy,Italy,188 cm,...,NaN,NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN
4,30996,M. Rohdén,Marcus Christer,Rohdén,32.0,1991-05-11,Värnamo,Sweden,Sweden,182 cm,...,NaN,NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1088,127011,A. Cambiaso,Andrea,Cambiaso,24.0,2000-02-20,Genoa,Italy,Italy,182 cm,...,20.0,35.0,10.0,0.0,0.0,None,None,0.0,0.0,NaN
1089,157052,R. Calafiori,Riccardo,Calafiori,22.0,2002-05-19,Roma,Italy,Italy,188 cm,...,27.0,35.0,4.0,0.0,0.0,None,None,0.0,0.0,NaN
1090,203474,N. Zalewski,Nicola,Zalewski,22.0,2002-01-23,Tivoli,Italy,Poland,164 cm,...,27.0,14.0,2.0,1.0,0.0,None,None,0.0,0.0,NaN
1091,289761,G. Scalvini,Giorgio,Scalvini,21.0,2003-12-11,Chiari,Italy,Italy,194 cm,...,10.0,46.0,3.0,0.0,0.0,None,None,0.0,0.0,NaN


In [95]:
players_df.tail()

,player_id,player_name,player_firstname,player_lastname,player_age,player_birth_date,player_birth_place,player_birth_country,player_nationality,player_height,...,fouls_drawn,fouls_committed,cards_yellow,cards_yellowred,cards_red,penalty_won,penalty_commited,penalty_scored,penalty_missed,penalty_saved
1088,297139,É. Camara,Étienne Amara,Camara,21.0,2003-03-30,Noisy-le-Grand,France,France,190 cm,...,NaN,NaN,0.0,0.0,0.0,None,None,0.0,0.0,NaN
1089,298242,F. Cissé,Faroukou Kayode,Cissé,20.0,2004-03-09,None,Côte d'Ivoire,Germany,None,...,NaN,NaN,0.0,0.0,0.0,None,None,0.0,0.0,NaN
1090,335071,M. Berisha,Medon,Berisha,21.0,2003-10-21,Münsingen,Switzerland,Albania,186 cm,...,5.0,7.0,0.0,0.0,0.0,None,None,0.0,0.0,NaN
1091,348533,S. Fini,Seydou,Fini,18.0,2006-06-02,None,Italy,Italy,178 cm,...,2.0,NaN,0.0,0.0,0.0,None,None,0.0,0.0,NaN
1092,375986,L. Kuavita,Léandre Filipe,Kuavita,20.0,2004-05-31,None,Belgium,Belgium,None,...,NaN,NaN,0.0,0.0,0.0,None,None,0.0,0.0,NaN


In [96]:
# Filtrare i giocatori che hanno almeno una apparizione in partita
players_with_appearances = players_df.query("games_appearences > 0")
players_with_appearances

,player_id,player_name,player_firstname,player_lastname,player_age,player_birth_date,player_birth_place,player_birth_country,player_nationality,player_height,...,fouls_drawn,fouls_committed,cards_yellow,cards_yellowred,cards_red,penalty_won,penalty_commited,penalty_scored,penalty_missed,penalty_saved
15,291780,D. Boloca,Daniel,Boloca,26.0,1998-12-22,Chieri,Italy,Romania,188 cm,...,7.0,25.0,6.0,0.0,1.0,None,None,0.0,0.0,NaN
17,315026,Y. Kallon,Yayah,Kallon,23.0,2001-06-30,None,Sierra Leone,Sierra Leone,175 cm,...,NaN,NaN,0.0,0.0,0.0,None,None,0.0,0.0,NaN
23,150,Arthur,Arthur Henrique,Ramos de Oliveira Melo,28.0,1996-08-12,Goiânia,Brazil,Brazil,171 cm,...,47.0,18.0,2.0,0.0,0.0,None,None,1.0,0.0,NaN
26,319,S. Luperto,Sebastiano,Luperto,28.0,1996-09-06,Lecce,Italy,Italy,191 cm,...,18.0,25.0,4.0,0.0,0.0,None,None,0.0,0.0,NaN
32,30490,N. Sansone,Nicola,Domenico Sansone,33.0,1991-09-10,München,Germany,Italy,175 cm,...,12.0,16.0,4.0,0.0,0.0,None,None,0.0,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1084,135519,G. Isaksen,Gustav,Tang Isaksen,23.0,2001-04-19,Hjerk,Denmark,Denmark,178 cm,...,24.0,17.0,3.0,0.0,0.0,None,None,0.0,0.0,NaN
1085,158710,V. Kristiansen,Victor,Bernth Kristiansen,22.0,2002-12-16,København,Denmark,Denmark,181 cm,...,7.0,29.0,4.0,0.0,0.0,None,None,0.0,0.0,NaN
1087,203474,N. Zalewski,Nicola,Zalewski,22.0,2002-01-23,Tivoli,Italy,Poland,164 cm,...,27.0,14.0,2.0,1.0,0.0,None,None,0.0,0.0,NaN
1090,335071,M. Berisha,Medon,Berisha,21.0,2003-10-21,Münsingen,Switzerland,Albania,186 cm,...,5.0,7.0,0.0,0.0,0.0,None,None,0.0,0.0,NaN


In [97]:
# Filtrare i giocatori il cui team_name in lowercase contiene "milan"
milan_players = players_df[players_df['team_name'].str.lower().str.contains('milan')]

# Visualizzare i giocatori filtrati
milan_players

,player_id,player_name,player_firstname,player_lastname,player_age,player_birth_date,player_birth_place,player_birth_country,player_nationality,player_height,...,fouls_drawn,fouls_committed,cards_yellow,cards_yellowred,cards_red,penalty_won,penalty_commited,penalty_scored,penalty_missed,penalty_saved
28,1831,A. Rebić,Ante,Rebić,30.0,1993-09-21,Split,Croatia,Croatia,185 cm,...,NaN,NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN
91,277175,A. Jungdal,Andreas Kristoffer,Jungdal,21.0,2002-02-22,Singapore,Singapore,Denmark,195 cm,...,NaN,NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN
103,81012,D. Vásquez,Devis Estiven,Vásquez Llach,25.0,1998-05-12,Barranquilla,Colombia,Colombia,195 cm,...,NaN,NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN
107,215976,N. Michelis,Nikolaos,Michelis,22.0,2001-03-23,Nea Smyrni,Greece,Greece,192 cm,...,NaN,NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN
111,343571,M. Malaspina,Mattia,Malaspina,18.0,2005-07-24,None,Italy,Italy,None,...,NaN,NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN
112,382947,H. Cuenca,Hugo Francisco,Cuenca Martínez,18.0,2005-01-08,None,Paraguay,Paraguay,183 cm,...,NaN,NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN
113,384419,D. Paloschi,Dorian,Paloschi,18.0,2005-11-21,None,Italy,Italy,None,...,NaN,NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN
114,384671,J. Longhi,Jordan,Longhi,18.0,2005-03-21,None,Italy,Italy,None,...,NaN,NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN
115,419617,V. Magni,Vittorio,Magni,17.0,2006-06-01,None,Italy,Italy,None,...,NaN,NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN
116,427335,D. Simic,Dario,Simic,NaN,None,None,Italy,Italy,None,...,NaN,NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN


### Giocatori attualmente in rosa in un Team

In [14]:
teams_df[["team_id","team_name"]]

,team_id,team_name
0,487,Lazio
1,488,Sassuolo
2,489,AC Milan
3,490,Cagliari
4,492,Napoli
5,494,Udinese
6,495,Genoa
7,496,Juventus
8,497,AS Roma
9,499,Atalanta


In [16]:
# Esempio AC Milan
team_players = call_api('players/squads', {'team': 489})
print(json.dumps(team_players, indent=4))

{
    "get": "players/squads",
    "parameters": {
        "team": "489"
    },
    "errors": [],
    "results": 1,
    "paging": {
        "current": 1,
        "total": 1
    },
    "response": [
        {
            "team": {
                "id": 489,
                "name": "AC Milan",
                "logo": "https://media.api-sports.io/football/teams/489.png"
            },
            "players": [
                {
                    "id": 22221,
                    "name": "M. Maignan",
                    "age": 28,
                    "number": 16,
                    "position": "Goalkeeper",
                    "photo": "https://media.api-sports.io/football/players/22221.png"
                },
                {
                    "id": 31069,
                    "name": "M. Sportiello",
                    "age": 31,
                    "number": 57,
                    "position": "Goalkeeper",
                    "photo": "https://media.api-sports.io/football/players